# Explore here

In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

import tensorflow as tf
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.datasets import mnist

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score


import warnings

2025-02-04 15:37:32.587840: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-02-04 15:37:32.602184: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-02-04 15:37:33.482197: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-02-04 15:37:33.677365: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1738683454.041337     817 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1738683454.14

In [15]:
import os

ruta = "/workspaces/Clasificador_de_Imagenes/data/imagenes_train"

if os.path.exists(ruta):
    print("✅ La ruta existe")
else:
    print("❌ La ruta NO existe")


❌ La ruta NO existe


In [14]:
import os
import matplotlib.pyplot as plt
from PIL import Image

# Rutas de las carpetas de entrenamiento y prueba
train_dir = "/workspaces/Clasificador_de_Imagenes/data/imagenes_train"
test_dir = "/workspaces/Clasificador_de_Imagenes/data/test1"

# Función para cargar imágenes de una carpeta
def load_images_from_folder(folder, label, max_images=9):
    images = []
    count = 0
    for filename in os.listdir(folder):
        if label in filename:  # Filtra por perros o gatos
            img_path = os.path.join(folder, filename)
            img = Image.open(img_path)
            images.append(img)
            count += 1
            if count >= max_images:
                break
    return images

# Cargar imágenes de perros y gatos
dog_images = load_images_from_folder(train_dir, "dog")
cat_images = load_images_from_folder(train_dir, "cat")

# Función para mostrar 9 imágenes
def plot_images(images, title):
    plt.figure(figsize=(10, 10))
    for i, img in enumerate(images):
        plt.subplot(3, 3, i + 1)
        plt.imshow(img)
        plt.axis("off")
    plt.suptitle(title, fontsize=16)
    plt.show()

# Mostrar 9 imágenes de perros y gatos
plot_images(dog_images, "Imágenes de Perros")
plot_images(cat_images, "Imágenes de Gatos")

FileNotFoundError: [Errno 2] No such file or directory: '/workspaces/Clasificador_de_Imagenes/data/imagenes_train'